# High-level Comparison Between the Network Covariance and Characteristic Matrices

In this notebook, we provide a rough comparison between covariance-based and entropy-based topology inference methods.
The goal is to test each method across a range of entangled states to get an idea of their relative performance.


## Key Differences

### Covariance Matrix Advantages

There is a significant performance advantage over the characteristic matrix:
* This can easily be observed by noting the cell execution times, especially when more qubits are considered. 
* The covariance is evaluated from one circuit evaluation $O(1)$, whereas the charcteristic matrix requires a minimum of $O(n^2)$ circuit evaluations where $n$ is the number of qubits. Given clever parallelization in quantum circuits, the characteristic matrix may be evaluated in $O(n)$.
* Optimizing the covariance matrix seems to require significantly fewer steps than needed to successfully infer topology using the characteristic matrix. This means that fewer circuits need to be run.
* The optimization appears to be more robust and less susceptible to local minima. We see this in the fact that the characteristic matrix often fails to infer the GHZ states with more than 3 qubits.


### Characteristic Matrix Advantages

A key difference between the covariance and characteristic matrices is that the covariance matrix seems to always have ones along the diagonal, even if the measured qubit is not correlated with any other qubits. On the contrary, the characteristic matrix measures a zero von neumann entropy if there is no correlation whatsoever.

In [1]:
import qnetti
import pennylane as qml
from pennylane import numpy as qnp
import qnetvo

print(qnetvo.__version__)
print(qml.__version__)

0.4.1
0.28.0


In [2]:
def decision_matrix(mat, atol=0.1):
    return qnp.where(qnp.abs(mat) > atol, 1, 0)


def matrix_distance(mat1, mat2):
    return qnp.linalg.norm(qnp.array(mat1) - qnp.abs(qnp.array(mat2)))


def characteristic_matrix_inference(prep_node, expected_mat, **kwargs):
    char_mat = qnetti.qubit_characteristic_matrix(prep_node, **kwargs)
    dec_mat = decision_matrix(char_mat)
    char_dist = matrix_distance(expected_mat, char_mat)

    print("characteristic matrix :\n", char_mat)
    print("decision matrix :\n", dec_mat)
    print("expected matrix :\n", expected_mat)
    print("distance to expected : ", char_dist)


def covariance_matrix_inference(
    prep_node, num_qubits, expected_mat,
    meas_wires=None, step_size=0.1, num_steps=10, verbose=False
):
    opt_dict = qnetvo.gradient_descent(
        qnetti.qubit_covariance_cost_fn(prep_node, meas_wires=meas_wires),
        qnp.random.rand(3 * num_qubits, requires_grad=True),
        step_size=step_size,
        num_steps=num_steps,
        sample_width=1,
        verbose=False,
    )

    cov_mat = qnetti.qubit_covariance_matrix_fn(prep_node, meas_wires=meas_wires)(
        opt_dict["opt_settings"]
    )
    dec_mat = decision_matrix(cov_mat)
    cov_dist = matrix_distance(expected_mat, cov_mat)

    print("covariance matrix :\n", cov_mat)
    print("decision matrix :\n", dec_mat)
    print("expected matrix :\n", expected_mat)
    print("distance to expected : ", cov_dist)

## Two-Qubit GHZ State

In [3]:
ghz2_mat = qnp.ones((2, 2))


def rot_bell_state(settings, wires):
    qnetvo.ghz_state(settings, wires)
    qml.ArbitraryUnitary([qnp.pi / 4, 0.3, -1.5], wires=[wires[0]])
    qml.ArbitraryUnitary([-0.7, 2.9, 1.2], wires=[wires[1]])


bell_state_prep_node = qnetvo.PrepareNode(wires=[0, 1], ansatz_fn=rot_bell_state)

In [4]:
characteristic_matrix_inference(bell_state_prep_node, ghz2_mat, num_steps=10, step_size=0.1)

characteristic matrix :
 [[1.         0.81074427]
 [0.81074427 1.        ]]
decision matrix :
 [[1 1]
 [1 1]]
expected matrix :
 [[1. 1.]
 [1. 1.]]
distance to expected :  0.26764802420514416


In [5]:
covariance_matrix_inference(
    bell_state_prep_node,
    2,
    ghz2_mat,
    step_size=0.2,
    num_steps=6,
    verbose=False,
)

covariance matrix :
 [[1.         0.99999526]
 [0.99999526 1.        ]]
decision matrix :
 [[1 1]
 [1 1]]
expected matrix :
 [[1. 1.]
 [1. 1.]]
distance to expected :  6.7081664685343485e-06


## 3-Qubit GHZ state

In [6]:
ghz3_mat = qnp.ones((3, 3))


def rot_ghz3_state(settings, wires):
    qnetvo.ghz_state(settings, wires)
    qml.ArbitraryUnitary([qnp.pi / 4, 0.3, -1.5], wires=[wires[0]])
    qml.ArbitraryUnitary([-0.7, 2.9, 1.2], wires=[wires[1]])
    qml.ArbitraryUnitary([-0.9, 0.9, 0.12], wires=[wires[2]])


ghz3_state_prep_node = qnetvo.PrepareNode(wires=[0, 1, 2], ansatz_fn=rot_ghz3_state)

In [7]:
characteristic_matrix_inference(ghz3_state_prep_node, ghz3_mat, num_steps=10, step_size=0.1)

characteristic matrix :
 [[1.         0.28656711 0.01555671]
 [0.28656711 1.         0.15772751]
 [0.01555671 0.15772751 1.        ]]
decision matrix :
 [[1 1 0]
 [1 1 1]
 [0 1 1]]
expected matrix :
 [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
distance to expected :  2.0916682418521884


In [8]:
covariance_matrix_inference(
    ghz3_state_prep_node,
    3,
    ghz3_mat,
    step_size=0.1,
    num_steps=10,
    verbose=False,
)

covariance matrix :
 [[1.         0.96365081 0.99935153]
 [0.96365081 1.         0.96423591]
 [0.99935153 0.96423591 1.        ]]
decision matrix :
 [[1 1 1]
 [1 1 1]
 [1 1 1]]
expected matrix :
 [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
distance to expected :  0.07212148529391496


## 4-Qubit GHZ

In [9]:
ghz4_mat = qnp.ones((4, 4))


def rot_ghz4_state(settings, wires):
    qnetvo.ghz_state(settings, wires)
    qml.ArbitraryUnitary([qnp.pi / 4, 0.3, -1.5], wires=[wires[0]])
    qml.ArbitraryUnitary([-0.7, 2.9, 1.2], wires=[wires[1]])
    qml.ArbitraryUnitary([-0.9, 0.9, 0.12], wires=[wires[2]])
    qml.ArbitraryUnitary([2, 2, 2], wires=[wires[3]])


ghz4_state_prep_node = qnetvo.PrepareNode(wires=[0, 1, 2, 3], ansatz_fn=rot_ghz4_state)

In [10]:
characteristic_matrix_inference(ghz4_state_prep_node, ghz4_mat, num_steps=30, step_size=0.1)

characteristic matrix :
 [[1.         0.12733596 0.99634087 0.95663169]
 [0.12733596 1.         0.62061772 0.9351972 ]
 [0.99634087 0.62061772 1.         0.45888047]
 [0.95663169 0.9351972  0.45888047 1.        ]]
decision matrix :
 [[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]
expected matrix :
 [[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
distance to expected :  1.552016353767203


In [11]:
covariance_matrix_inference(
    ghz4_state_prep_node,
    4,
    ghz4_mat,
    step_size=0.2,
    num_steps=6,
    verbose=False,
)

covariance matrix :
 [[ 1.         -0.94770159  0.9753671  -0.99221991]
 [-0.94770159  1.         -0.9283968   0.94443803]
 [ 0.9753671  -0.9283968   1.         -0.97200827]
 [-0.99221991  0.94443803 -0.97200827  1.        ]]
decision matrix :
 [[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]
expected matrix :
 [[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
distance to expected :  0.15748092700989463


# 5-Qubit GHZ

In [12]:
ghz5_mat = qnp.ones((5, 5))


def rot_ghz5_state(settings, wires):
    qnetvo.ghz_state(settings, wires)
    qml.ArbitraryUnitary([qnp.pi / 4, 0.3, -1.5], wires=[wires[0]])
    qml.ArbitraryUnitary([-0.7, 2.9, 1.2], wires=[wires[1]])
    qml.ArbitraryUnitary([-0.9, 0.9, 0.12], wires=[wires[2]])
    qml.ArbitraryUnitary([2, 2, 2], wires=[wires[3]])
    qml.ArbitraryUnitary([1, 2.3, 0.4], wires=[wires[4]])


ghz5_state_prep_node = qnetvo.PrepareNode(wires=[0, 1, 2, 3, 4], ansatz_fn=rot_ghz5_state)

In [13]:
characteristic_matrix_inference(ghz5_state_prep_node, ghz5_mat, num_steps=30, step_size=0.1)

characteristic matrix :
 [[1.         0.99994785 0.02779974 0.9998656  0.43104335]
 [0.99994785 1.         0.86284223 0.97404714 1.        ]
 [0.02779974 0.86284223 1.         0.00887094 0.12588957]
 [0.9998656  0.97404714 0.00887094 1.         0.97155213]
 [0.43104335 1.         0.12588957 0.97155213 1.        ]]
decision matrix :
 [[1 1 0 1 1]
 [1 1 1 1 1]
 [0 1 1 0 1]
 [1 1 0 1 1]
 [1 1 1 1 1]]
expected matrix :
 [[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
distance to expected :  2.463974830636028


In [15]:
covariance_matrix_inference(
    ghz5_state_prep_node, 5, ghz5_mat, step_size=0.1, num_steps=10, verbose=False
)

covariance matrix :
 [[ 1.          0.99995968  0.99999997 -0.99999986 -0.99999999]
 [ 0.99995968  1.          0.99995967 -0.99995956 -0.99995969]
 [ 0.99999997  0.99995967  1.         -0.99999985 -0.99999998]
 [-0.99999986 -0.99995956 -0.99999985  1.          0.99999987]
 [-0.99999999 -0.99995969 -0.99999998  0.99999987  1.        ]]
decision matrix :
 [[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]]
expected matrix :
 [[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
distance to expected :  0.0001141291785667427


# 8-qubit GHZ state

In [16]:
ghz8_mat = qnp.ones((8, 8))


def rot_ghz8_state(settings, wires):
    qnetvo.ghz_state(settings, wires)
    qml.ArbitraryUnitary([qnp.pi / 4, 0.3, -1.5], wires=[wires[0]])
    qml.ArbitraryUnitary([-0.7, 2.9, 1.2], wires=[wires[1]])
    qml.ArbitraryUnitary([-0.9, 0.9, 0.12], wires=[wires[2]])
    qml.ArbitraryUnitary([2, 2, 2], wires=[wires[3]])
    qml.ArbitraryUnitary([1, 2.3, 0.4], wires=[wires[4]])
    qml.ArbitraryUnitary([-0.9, 0.9, 0.12], wires=[wires[5]])
    qml.ArbitraryUnitary([2, 2, 2], wires=[wires[6]])
    qml.ArbitraryUnitary([1, 2.3, 0.4], wires=[wires[7]])


ghz8_state_prep_node = qnetvo.PrepareNode(
    wires=[0, 1, 2, 3, 4, 5, 6, 7],
    ansatz_fn=rot_ghz8_state,
)

In [17]:
characteristic_matrix_inference(ghz8_state_prep_node, ghz8_mat, num_steps=30, step_size=0.1)

characteristic matrix :
 [[1.00000000e+00 9.99999998e-01 2.07680546e-03 5.26347886e-01
  9.73173256e-01 3.31328899e-02 3.55829492e-01 1.10153174e-01]
 [9.99999998e-01 1.00000000e+00 1.00000000e+00 9.98713697e-01
  9.99999998e-01 7.86119230e-02 9.88349376e-01 9.99999996e-01]
 [2.07680546e-03 1.00000000e+00 1.00000000e+00 5.65422042e-01
  2.00212341e-01 4.48513594e-03 1.47067545e-03 5.05075414e-02]
 [5.26347886e-01 9.98713697e-01 5.65422042e-01 1.00000000e+00
  4.07563223e-04 9.72553105e-01 9.42003427e-01 4.34337588e-03]
 [9.73173256e-01 9.99999998e-01 2.00212341e-01 4.07563223e-04
  1.00000000e+00 1.51308573e-01 9.35393767e-01 8.55871639e-02]
 [3.31328899e-02 7.86119230e-02 4.48513594e-03 9.72553105e-01
  1.51308573e-01 1.00000000e+00 9.55916290e-01 3.45274249e-01]
 [3.55829492e-01 9.88349376e-01 1.47067545e-03 9.42003427e-01
  9.35393767e-01 9.55916290e-01 1.00000000e+00 5.72120042e-01]
 [1.10153174e-01 9.99999996e-01 5.05075414e-02 4.34337588e-03
  8.55871639e-02 3.45274249e-01 5.7212

In [18]:
covariance_matrix_inference(
    ghz8_state_prep_node,
    8,
    ghz8_mat,
    step_size=0.1,
    num_steps=10,
    verbose=False,
)

covariance matrix :
 [[ 1.          0.86668224  0.8317666  -0.81134972 -0.81585471  0.81661026
  -0.81637776 -0.81570523]
 [ 0.86668224  1.          0.86575385 -0.84450271 -0.84919177  0.8499782
  -0.8497362  -0.84903619]
 [ 0.8317666   0.86575385  1.         -0.81048061 -0.81498077  0.81573551
  -0.81550326 -0.81483146]
 [-0.81134972 -0.84450271 -0.81048061  1.          0.79497592 -0.79571214
   0.79548559  0.79483027]
 [-0.81585471 -0.84919177 -0.81498077  0.79497592  1.         -0.8001303
   0.79990249  0.79924354]
 [ 0.81661026  0.8499782   0.81573551 -0.79571214 -0.8001303   1.
  -0.80064327 -0.7999837 ]
 [-0.81637776 -0.8497362  -0.81550326  0.79548559  0.79990249 -0.80064327
   1.          0.79975594]
 [-0.81570523 -0.84903619 -0.81483146  0.79483027  0.79924354 -0.7999837
   0.79975594  1.        ]]
decision matrix :
 [[1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]]
expected 

# 8 qubit graph state

In [19]:
# This expected matrix is not known with certainty, but agrees with results
graph8_mat = qnp.array(
    [
        [1, 0, 0, 1, 0, 0, 0, 0],
        [0, 1, 1, 0, 0, 0, 0, 0],
        [0, 1, 1, 0, 0, 0, 0, 0],
        [1, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1],
    ]
)

graph8_state_prep_node = qnetvo.PrepareNode(
    wires=[0, 1, 2, 3, 4, 5, 6, 7],
    ansatz_fn=qnetvo.graph_state_fn([[0, 1], [1, 2], [0, 3], [6, 7]]),
)

In [20]:
characteristic_matrix_inference(graph8_state_prep_node, graph8_mat, num_steps=60, step_size=0.02)

characteristic matrix :
 [[1.00000000e+00 2.22044605e-15 2.88657986e-15 9.12678121e-01
  2.66453526e-15 2.22044605e-15 3.10862447e-15 2.44249065e-15]
 [2.22044605e-15 1.00000000e+00 9.93458745e-01 2.88657986e-15
  3.10862447e-15 2.88657986e-15 2.88657986e-15 4.44089210e-15]
 [2.88657986e-15 9.93458745e-01 1.00000000e+00 1.77635684e-15
  4.44089210e-15 3.10862447e-15 4.44089210e-15 4.44089210e-15]
 [9.12678121e-01 2.88657986e-15 1.77635684e-15 1.00000000e+00
  3.10862447e-15 2.66453526e-15 2.88657986e-15 2.22044605e-15]
 [2.66453526e-15 3.10862447e-15 4.44089210e-15 3.10862447e-15
  4.49639799e-09 4.21884749e-15 4.66293670e-15 1.33226763e-15]
 [2.22044605e-15 2.88657986e-15 3.10862447e-15 2.66453526e-15
  4.21884749e-15 8.46170799e-01 2.66453526e-15 2.22044605e-15]
 [3.10862447e-15 2.88657986e-15 4.44089210e-15 2.88657986e-15
  4.66293670e-15 2.66453526e-15 1.00000000e+00 1.97551191e-03]
 [2.44249065e-15 4.44089210e-15 4.44089210e-15 2.22044605e-15
  1.33226763e-15 2.22044605e-15 1.9755

In [21]:
covariance_matrix_inference(
    graph8_state_prep_node,
    8,
    graph8_mat,
    step_size=0.05,
    num_steps=30,
    verbose=False,
)

covariance matrix :
 [[ 1.00000000e+00 -3.33066907e-16  2.49800181e-16 -9.99994664e-01
   2.77555756e-17  8.32667268e-17  0.00000000e+00  5.55111512e-17]
 [-3.33066907e-16  1.00000000e+00 -9.99998972e-01  1.11022302e-16
  -1.10937023e-16  5.56379244e-17  5.55111512e-17  5.55111512e-17]
 [ 2.49800181e-16 -9.99998972e-01  1.00000000e+00 -1.11022302e-16
   1.10937023e-16  5.53843781e-17 -5.55111512e-17 -5.55111512e-17]
 [-9.99994664e-01  1.11022302e-16 -1.11022302e-16  1.00000000e+00
  -2.77982156e-17  5.54477647e-17  5.55111512e-17 -8.32667268e-17]
 [ 2.77555756e-17 -1.10937023e-16  1.10937023e-16 -2.77982156e-17
   9.99999410e-01  8.84336278e-17  4.26399770e-20  5.55111512e-17]
 [ 8.32667268e-17  5.56379244e-17  5.53843781e-17  5.54477647e-17
   8.84336278e-17  9.99998696e-01 -1.10958916e-16  5.55111512e-17]
 [ 0.00000000e+00  5.55111512e-17 -5.55111512e-17  5.55111512e-17
   4.26399770e-20 -1.10958916e-16  1.00000000e+00 -9.99999890e-01]
 [ 5.55111512e-17  5.55111512e-17 -5.55111512e-1

## 8-Qubit Graph State with Frustrated CNOT arrangement

In [22]:
graph8_cyclic_mat = qnp.array(
    [
        [1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1],
    ]
)

graph8_cyclic_state_prep_node = qnetvo.PrepareNode(
    wires=[0, 1, 2, 3, 4, 5, 6, 7],
    ansatz_fn=qnetvo.graph_state_fn([[0, 1], [1, 2], [2, 0], [6, 7]]),
)

In [23]:
characteristic_matrix_inference(
    graph8_cyclic_state_prep_node, graph8_cyclic_mat, num_steps=60, step_size=0.02
)

characteristic matrix :
 [[1.00000000e+00 6.86737674e-01 2.56474166e-04 2.22044605e-15
  3.10862447e-15 2.66453526e-15 3.77475828e-15 3.10862447e-15]
 [6.86737674e-01 1.00000000e+00 8.26111627e-03 2.66453526e-15
  2.88657986e-15 3.33066907e-15 2.88657986e-15 2.88657986e-15]
 [2.56474166e-04 8.26111627e-03 1.00000000e+00 2.22044605e-15
  2.88657986e-15 3.55271368e-15 3.55271368e-15 3.55271368e-15]
 [2.22044605e-15 2.66453526e-15 2.22044605e-15 2.60175877e-05
  1.99840144e-15 2.55351296e-15 2.66453526e-15 3.10862447e-15]
 [3.10862447e-15 2.88657986e-15 2.88657986e-15 1.99840144e-15
  5.70703484e-09 2.22044605e-15 3.99680289e-15 4.44089210e-15]
 [2.66453526e-15 3.33066907e-15 3.55271368e-15 2.55351296e-15
  2.22044605e-15 7.16166655e-02 3.33066907e-15 2.66453526e-15]
 [3.77475828e-15 2.88657986e-15 3.55271368e-15 2.66453526e-15
  3.99680289e-15 3.33066907e-15 1.00000000e+00 9.98354407e-01]
 [3.10862447e-15 2.88657986e-15 3.55271368e-15 3.10862447e-15
  4.44089210e-15 2.66453526e-15 9.9835

In [24]:
covariance_matrix_inference(
    graph8_cyclic_state_prep_node,
    8,
    graph8_cyclic_mat,
    step_size=0.05,
    num_steps=30,
    verbose=False,
)

covariance matrix :
 [[ 1.00000000e+00  9.99488361e-01  9.99934385e-01  1.11563637e-16
   2.79572630e-17  8.40862465e-17  8.32667268e-17 -5.55111512e-17]
 [ 9.99488361e-01  1.00000000e+00  9.99461170e-01  1.11671904e-16
   2.42024850e-19 -5.45277277e-17  8.32667268e-17 -5.55111512e-17]
 [ 9.99934385e-01  9.99461170e-01  1.00000000e+00  1.67074788e-16
  -2.75538882e-17  8.19519605e-19  8.32667268e-17 -5.55111512e-17]
 [ 1.11563637e-16  1.11671904e-16  1.67074788e-16  9.99996196e-01
   1.16253154e-17  4.62818802e-17 -5.56194181e-17  1.08266912e-19]
 [ 2.79572630e-17  2.42024850e-19 -2.75538882e-17  1.16253154e-17
   9.99999472e-01 -1.72296242e-16 -5.55514887e-17 -2.77152381e-17]
 [ 8.40862465e-17 -5.45277277e-17  8.19519605e-19  4.62818802e-17
  -1.72296242e-16  9.99991282e-01  2.75916717e-17  5.56750552e-17]
 [ 8.32667268e-17  8.32667268e-17  8.32667268e-17 -5.56194181e-17
  -5.55514887e-17  2.75916717e-17  1.00000000e+00 -9.99999878e-01]
 [-5.55111512e-17 -5.55111512e-17 -5.55111512e-1

## 12-Qubit Graph State

In [25]:
graph12_mat = qnp.array(
    [
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
    ]
)
graph12_state_prep_node = qnetvo.PrepareNode(
    wires=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    ansatz_fn=qnetvo.graph_state_fn([[0, 1], [1, 2], [6, 7], [4, 5], [4, 9], [4, 11], [11, 10]]),
)

In [26]:
characteristic_matrix_inference(graph12_state_prep_node, graph12_mat, num_steps=60, step_size=0.02)

characteristic matrix :
 [[ 1.00000000e+00  6.63487149e-03  9.85590457e-01  4.21884749e-15
   4.44089210e-15  4.44089210e-15  2.88657986e-15  3.10862447e-15
   5.55111512e-15  1.77635684e-15  4.44089210e-15  1.33226763e-15]
 [ 6.63487149e-03  1.00000000e+00  2.18029429e-01  2.88657986e-15
   2.22044605e-15  4.44089210e-15  4.44089210e-15  3.99680289e-15
   3.99680289e-15  6.66133815e-15  6.66133815e-16  1.48769885e-14]
 [ 9.85590457e-01  2.18029429e-01  1.00000000e+00  3.33066907e-15
   4.44089210e-15  4.88498131e-15  4.44089210e-15  3.10862447e-15
   2.22044605e-15  2.88657986e-15  2.17603713e-14  4.66293670e-15]
 [ 4.21884749e-15  2.88657986e-15  3.33066907e-15  8.92165330e-09
   4.21884749e-15  3.55271368e-15  3.10862447e-15  3.77475828e-15
   5.55111512e-16  7.32747196e-15  6.43929354e-15  2.66453526e-15]
 [ 4.44089210e-15  2.22044605e-15  4.44089210e-15  4.21884749e-15
   1.00000000e+00  4.39475833e-03  4.88498131e-15  2.66453526e-15
   0.00000000e+00  1.45896623e-02  5.77315973e-

In [27]:
covariance_matrix_inference(
    graph12_state_prep_node,
    12,
    graph12_mat,
    num_steps=20,
    step_size=0.05,
    verbose=False,
)

covariance matrix :
 [[ 1.00000000e+00 -9.99999989e-01  9.99999973e-01  2.77555756e-17
  -5.55111512e-17  5.55111512e-17  2.77555756e-17  2.77555756e-17
   2.22044605e-16  2.77555756e-17  1.38777878e-16 -2.77555756e-17]
 [-9.99999989e-01  1.00000000e+00 -9.99999972e-01 -5.55111512e-17
   0.00000000e+00  0.00000000e+00  5.55111512e-17  2.77555756e-17
  -1.66533454e-16 -2.77555756e-17 -1.66533454e-16 -2.77555756e-17]
 [ 9.99999973e-01 -9.99999972e-01  1.00000000e+00  5.55111512e-17
  -5.55111512e-17  5.55111512e-17  2.77555756e-17  2.77555756e-17
   6.10622664e-16  2.77555756e-17  1.38777878e-16 -2.77555756e-17]
 [ 2.77555756e-17 -5.55111512e-17  5.55111512e-17  9.99076076e-01
   3.37464724e-18 -7.82047560e-17 -5.71984749e-17 -2.26936048e-17
  -2.65426244e-17  0.00000000e+00 -1.43839849e-16 -1.68979175e-16]
 [-5.55111512e-17  0.00000000e+00 -5.55111512e-17  3.37464724e-18
   1.00000000e+00 -9.99997547e-01 -2.77555756e-17  8.32667268e-17
  -1.38654557e-16 -9.99993695e-01  1.66533454e-16  

## A bunch of different entangled states

In [28]:
multi_ent_mat_full = qnp.zeros((8, 8))
multi_ent_mat = qnp.array(
    [
        [1, 1, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 0, 0, 0],
        [0, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 1, 1, 1],
    ]
)


def fn(settings, wires):
    qnetvo.shared_coin_flip_state([qnp.pi / 3], wires=[wires[0], wires[1], wires[7]])
    qnetvo.ghz_state([], wires=wires[2:4])
    qnetvo.W_state([], wires=wires[4:7])


multi_ent_prep_node = qnetvo.PrepareNode(
    wires=[0, 1, 2, 3, 4, 5, 6, 7],
    ansatz_fn=fn,
)

In [29]:
characteristic_matrix_inference(
    multi_ent_prep_node, multi_ent_mat_full, num_steps=50, step_size=0.1
)

characteristic matrix :
 [[8.21844448e-01 2.83082972e-01 1.33226763e-15 1.33226763e-15
  1.77635684e-15 8.88178420e-16 2.44249065e-15 8.11278124e-01]
 [2.83082972e-01 9.42812328e-01 1.33226763e-15 1.33226763e-15
  1.33226763e-15 1.11022302e-15 1.77635684e-15 8.11278125e-01]
 [1.33226763e-15 1.33226763e-15 1.00000000e+00 1.00000000e+00
  2.88657986e-15 1.55431223e-15 1.55431223e-15 2.22044605e-15]
 [1.33226763e-15 1.33226763e-15 1.00000000e+00 1.00000000e+00
  1.11022302e-15 1.77635684e-15 1.55431223e-15 1.99840144e-15]
 [1.77635684e-15 1.33226763e-15 2.88657986e-15 1.11022302e-15
  9.99994355e-01 3.48930784e-01 3.30417854e-01 1.33226763e-15]
 [8.88178420e-16 1.11022302e-15 1.55431223e-15 1.77635684e-15
  3.48930784e-01 9.92267422e-01 3.49874615e-01 1.33226763e-15]
 [2.44249065e-15 1.77635684e-15 1.55431223e-15 1.55431223e-15
  3.30417854e-01 3.49874615e-01 9.26284791e-01 8.88178420e-16]
 [8.11278124e-01 8.11278125e-01 2.22044605e-15 1.99840144e-15
  1.33226763e-15 1.33226763e-15 8.8817

In [31]:
covariance_matrix_inference(
    multi_ent_prep_node,
    7,
    multi_ent_mat,
    meas_wires=[0, 1, 2, 3, 4, 5, 6],
    num_steps=20,
    step_size=0.05,
    verbose=False,
)

covariance matrix :
 [[ 9.23814622e-01  2.02564941e-01  0.00000000e+00  0.00000000e+00
  -1.38777878e-17 -7.28583860e-17  2.60208521e-17]
 [ 2.02564941e-01  9.40156853e-01  0.00000000e+00  0.00000000e+00
  -1.73472348e-17 -5.89805982e-17  4.51028104e-17]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  9.99999979e-01
  -1.11022302e-16 -2.77555756e-17 -1.38777878e-16]
 [ 0.00000000e+00  0.00000000e+00  9.99999979e-01  1.00000000e+00
  -1.11022302e-16 -2.77555756e-17 -1.38777878e-16]
 [-1.38777878e-17 -1.73472348e-17 -1.11022302e-16 -1.11022302e-16
   9.98503232e-01  6.52703407e-01  6.57352698e-01]
 [-7.28583860e-17 -5.89805982e-17 -2.77555756e-17 -2.77555756e-17
   6.52703407e-01  9.98713890e-01  6.58621939e-01]
 [ 2.60208521e-17  4.51028104e-17 -1.38777878e-16 -1.38777878e-16
   6.57352698e-01  6.58621939e-01  9.99658292e-01]]
decision matrix :
 [[1 1 0 0 0 0 0]
 [1 1 0 0 0 0 0]
 [0 0 1 1 0 0 0]
 [0 0 1 1 0 0 0]
 [0 0 0 0 1 1 1]
 [0 0 0 0 1 1 1]
 [0 0 0 0 1 1 1]]
expected matrix :
 [